In [1]:
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

In [2]:
st.set_page_config(
    page_title="Land Restoration in Assaba, Sahel",
    page_icon="🏜️💦",
    layout="wide",
    initial_sidebar_state="expanded")

alt.themes.enable("dark")

2025-03-29 02:54:55.307 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ThemeRegistry.enable('dark')

In [7]:
df = pd.read_csv('clean_data.csv', index_col=0)
df

,year,lat,lon,gpp,land,pop,popdens,rainfall,dist,road,water,actual_lat,actual_lon
475,2010,0,475,0.0,16.0,0.000000,0.776286,0.0,1.0,0.0,0.0,15.159763,-10.799267
1039,2010,1,474,0.0,16.0,0.682958,0.709646,0.0,1.0,0.0,1.0,15.163930,-10.803797
1040,2010,1,475,0.0,16.0,0.000000,0.703225,0.0,1.0,0.0,0.0,15.163930,-10.799480
1603,2010,2,473,0.0,16.0,0.675067,0.693418,0.0,1.0,0.0,1.0,15.168097,-10.808327
1604,2010,2,474,0.0,16.0,0.677053,0.668584,0.0,1.0,0.0,0.0,15.168097,-10.804009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125410,2023,766,452,1378.0,10.0,0.000000,0.000000,0.0,1.0,0.0,1.0,18.351429,-11.082932
3125411,2023,767,449,2433.0,10.0,0.000000,0.000000,0.0,1.0,0.0,0.0,18.355597,-11.096370
3125412,2023,767,450,2450.0,10.0,0.000000,0.000000,0.0,1.0,0.0,0.0,18.355597,-11.091979
3125413,2023,767,451,2079.0,10.0,0.000000,0.000000,0.0,1.0,0.0,1.0,18.355597,-11.087589


In [8]:
with st.sidebar:
    st.title('🏜️💦 Land Restoration in Assaba, Sahel')
    
    year_list = list(df.year.unique())[::-1]
    
    selected_year = st.selectbox('Select a year', year_list, index=len(year_list)-1)
    df_selected_year = df[df.year == selected_year]
    df_selected_year_sorted = df_selected_year.sort_values(by="gpp", ascending=False)

    color_theme_list = ['blues', 'cividis', 'greens', 'inferno', 'magma', 'plasma', 'reds', 'rainbow', 'turbo', 'viridis']
    selected_color_theme = st.selectbox('Select a color theme', color_theme_list)

2025-03-29 03:03:41.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03:41.850 Session state does not function when running a script without `streamlit run`
2025-03-29 03:03:41.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 03:03

In [9]:
def make_heatmap(input_df, input_y, input_x, input_color, input_color_theme):
    heatmap = alt.Chart(input_df).mark_rect().encode(
            y=alt.Y(f'{input_y}:O', axis=alt.Axis(title="Year", titleFontSize=18, titlePadding=15, titleFontWeight=900, labelAngle=0)),
            x=alt.X(f'{input_x}:O', axis=alt.Axis(title="", titleFontSize=18, titlePadding=15, titleFontWeight=900)),
            color=alt.Color(f'max({input_color}):Q',
                             legend=None,
                             scale=alt.Scale(scheme=input_color_theme)),
            stroke=alt.value('black'),
            strokeWidth=alt.value(0.25),
        ).properties(width=900
        ).configure_axis(
        labelFontSize=12,
        titleFontSize=12
        ) 
    # height=300
    return heatmap

In [10]:
def make_choropleth(input_df, input_id, input_column, input_color_theme):
    choropleth = px.choropleth(input_df, locations=input_id, color=input_column, locationmode="USA-states",
                               color_continuous_scale=input_color_theme,
                               range_color=(0, max(df_selected_year.population)),
                               scope="usa",
                               labels={'population':'Population'}
                              )
    choropleth.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=350
    )
    return choropleth